# PipeLine ETL

## 1. Conexión a MySQL

Imports

In [1]:
import mysql.connector
import pandas as pd

Conexion a SQL

In [3]:
con = mysql.connector.connect(user = "root", password = "1234", host = "127.0.0.1")
cursor = con.cursor()

# 2. Crear Base de Datos y Tablas

In [7]:
cursor.execute("DROP DATABASE f1_db")

In [9]:
cursor.execute("CREATE DATABASE IF NOT EXISTS f1_db")


In [10]:
cursor.execute("USE f1_db")

Función auxiliar para show * tables

In [11]:
def mostrar_sql(sentencia_sql):
    cursor.execute(sentencia_sql)
    rows = cursor.fetchall()
    for row in rows:
        print(row)

### Tabla Piloto

In [12]:
sql = """
CREATE TABLE PILOTO (
	driverId INT PRIMARY KEY,
    driverRef VARCHAR(50),
    driverNumber INT,
    driverCode VARCHAR(3),
    forename VARCHAR(50),
    surname VARCHAR(50),
    dob DATE,
    nationality VARCHAR(50),
    url VARCHAR(255)
    );
"""
cursor.execute(sql)

In [13]:
sentencia_sql = "SHOW TABLES;"
mostrar_sql(sentencia_sql)

('piloto',)


### Tabla Equipo

In [14]:
sql = """
CREATE TABLE EQUIPO(
	constructorId INT PRIMARY KEY,
    constructorRef VARCHAR(50),
    constructorName VARCHAR(70),
    nationality VARCHAR(50),
    url VARCHAR(255)
    );
"""
cursor.execute(sql)

In [15]:
sentencia_sql = "SHOW TABLES;"
mostrar_sql(sentencia_sql)

('equipo',)
('piloto',)


### Tabla Circuito

In [16]:
sql = """
CREATE TABLE CIRCUITO(
	circuitId INT PRIMARY KEY,
    circuitRef VARCHAR(50),
    circuitName VARCHAR(70),
    location VARCHAR(50),
    country VARCHAR(50),
    lat DECIMAL(10, 6),
    lng DECIMAL(10, 6),
    alt INT,
    url VARCHAR(255)
    );
"""
cursor.execute(sql)

In [17]:
sentencia_sql = "SHOW TABLES;"
mostrar_sql(sentencia_sql)

('circuito',)
('equipo',)
('piloto',)


### Tabla Carrera

In [18]:
sql = """
CREATE TABLE CARRERA(
	raceId INT PRIMARY KEY,
    raceYear INT,
    raceRound INT,
    circuitId INT,
    FOREIGN KEY (circuitId) REFERENCES CIRCUITO(circuitId),
    raceName VARCHAR(100),
    raceDate DATE,
    raceTime TIME,
    url VARCHAR(255)
    );
"""
cursor.execute(sql)

In [19]:
sentencia_sql = "SHOW TABLES;"
mostrar_sql(sentencia_sql)

('carrera',)
('circuito',)
('equipo',)
('piloto',)


### Tabla Resultado

In [20]:
sql = """
CREATE TABLE RESULTADO(
	resultId INT PRIMARY KEY,
    raceId INT,
    FOREIGN KEY (raceId) REFERENCES CARRERA(raceId),
    driverId INT,
    FOREIGN KEY (driverId) REFERENCES PILOTO(driverId),
    constructorId INT,
	FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId),
    grid INT,
    positionResult INT,
    points DECIMAL(6,2),
    laps INT,
    totalTime VARCHAR(50)
    );
"""
cursor.execute(sql)

In [21]:
sentencia_sql = "SHOW TABLES;"
mostrar_sql(sentencia_sql)

('carrera',)
('circuito',)
('equipo',)
('piloto',)
('resultado',)


### Tabla Clasificación

In [22]:
sql = """
CREATE TABLE CLASIFICACION(
	qualifyingId INT PRIMARY KEY,
	raceId INT,
	FOREIGN KEY (raceId) REFERENCES CARRERA(raceId),
    driverId INT,
    FOREIGN KEY (driverId) REFERENCES PILOTO(driverId),
    constructorId INT,
	FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId),
	qualifyingPosition INT,
    q1 VARCHAR(50),
    q2 VARCHAR(50),
    q3 VARCHAR(50)
    );
""" 
cursor.execute(sql) 

In [23]:
sentencia_sql = "SHOW TABLES;"
mostrar_sql(sentencia_sql)

('carrera',)
('circuito',)
('clasificacion',)
('equipo',)
('piloto',)
('resultado',)


### Tabla info_circuito (scraping)


In [24]:
sql = """
CREATE TABLE INFO_CIRCUITO(
    circuitId INT PRIMARY KEY,  
    FOREIGN KEY (circuitId) REFERENCES CIRCUITO(circuitId),
    capacity VARCHAR(50),
    opened_year INT,            
    length VARCHAR(50),         
    turns INT
);
"""
cursor.execute(sql)

In [25]:
sentencia_sql = "SHOW TABLES;"
mostrar_sql(sentencia_sql)

('carrera',)
('circuito',)
('clasificacion',)
('equipo',)
('info_circuito',)
('piloto',)
('resultado',)


### Tabla info_equipo (scraping)

In [26]:
sql = """
CREATE TABLE INFO_EQUIPO(
	constructorId INT PRIMARY KEY,
    FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId),
    championships INT,
    race_wins INT,
    pole_positions INT,
    fastest_laps INT 
    );
"""
cursor.execute(sql)

In [27]:
sentencia_sql = "SHOW TABLES;"
mostrar_sql(sentencia_sql)

('carrera',)
('circuito',)
('clasificacion',)
('equipo',)
('info_circuito',)
('info_equipo',)
('piloto',)
('resultado',)


## 3. Limpieza

### 3.1. Funciones auxiliares

3.1.1. Limpiar capacity (circuito)

In [30]:
def limpiar_capacity(valor):
    #Si el valor es nulo o está vacío
    if pd.isna(valor) or valor == "":
        return None
    
    # Pasar el valor a string
    valor = str(valor)

    # Si hay texto que no indica número
    if "open seating" in valor.lower():
        return None
    
    # Recorrer caracter por caracter y sacar el primer número
    numero = ""
    digito_encontrado = False

    for char in valor:
        if char.isdigit():
            numero += char
            digito_encontrado = True
        elif char == "," and digito_encontrado:
            # Ignorar esta coma, es parte del número, ej: 125,000
            continue
        elif digito_encontrado:
            # Ya tenemos digito y encuentra otra cosa distinta
            break

    # Si encontramos número, pasar a int
    if numero:
        return int(numero)
    
    return None

3.1.2. Limpiar opened_year (circuito)

In [ ]:
def limpiar_opened_year(valor):
    # Si el avlor es nulo o está vacío
    if pd.isna(valor) or valor == "":
        return None

    # Pasar el valor a string
    valor = str(valor)

    # Buscar el primer grupo de 4 dígitos
    digitos = ""

    for char in valor:
        if char.isdigit():
            digitos += char
            if len(digitos) == 4:
                # comprobar si es un año válido
                anio = int (digitos)
                if 1900 <= anio <= 2030:
                    return anio
                else:
                # El año no era válido, hay que seguir buscando
                    digitos = ""

    return None

3.1.3. Limpiar length (circuito)

In [55]:
# En esta funcion voy a buscar donde está "km" y voy a sacar el valor que esté justo antes
def limpiar_length(valor):
    # Si el valor es nulo o está vacío
    if pd.isna(valor) or valor == "":
        return None

    # Pasar el valor a string
    valor = str(valor)

    pos_km = valor.lower().find("km")

    if pos_km == -1: 
        return None

    
    parte_antes = valor[:pos_km]


    numero = ""
    dentro_corchete = False

    for char in reversed(parte_antes):
        if char == "]":
            dentro_corchete = True
            continue
        
        elif char == "[":
            dentro_corchete = False
            continue

        elif dentro_corchete:
            continue

        if char.isdigit() or char == ".":
            numero = char + numero
        elif numero:
            #Ya encontré numero y hay otra cosa
            break


    #Convertir a float
    if numero and numero != ".":
        return float(numero)

    return None

    

3.1.4. Limpiar turns (circuito)

In [53]:
def limpiar_turns(valor):
    # Si el valor es nulo o está vacío
    if pd.isna(valor) or valor == "":
        return None

    # Pasar el valor a string
    valor = str(valor)

    # Sacar el priemr numero
    numero = ""

    for char in valor:
        if char.isdigit():
            numero += char
        elif numero:
            break

    if numero:
        return int(numero)
    
    return None

3.1.5. Limpiar valor numérico (general)

In [57]:
def limpiar_valor_numerico(valor):
    # convertir NaN a None y float a int si es posible
    if pd.isna(valor):
        return None

    if float(valor) == int(valor):
        return int (valor)
    
    return valor